<a href="https://colab.research.google.com/github/cokoroafor11/Playlist-Recommender-App/blob/main/Spotify_Playlist_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Imports

In [4]:
!pip install spotipy --upgrade
import spotipy
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.3/240.3 kB 12.4 MB/s eta 0:00:00


Create the Spotify object

In [5]:
#Save client credentials
client_id = "c1f74565be774e65aa211462aaf5fed8"
client_secret = "2edce4052f8f46639c0e112658572d66"
#Create object
spotify = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= client_id,client_secret=client_secret),requests_timeout=100,retries=3)

I need to extract songs from a playlist

In [1]:
def extract_songs(playlist):
    '''Function to extract songs from a playlist: Will run this for all the playlists being added'''

    #Make sure playlist input is string
    if type(playlist) != str: 
        playlist = str(playlist)

    #Get playlist tracks    
    playlist_tracks =  spotify.playlist_tracks(playlist)
    tracks = []

    #Append tracks to list if no type error
    for elem in playlist_tracks['items']:
        try:
            tracks.append(elem['track']['external_urls']['spotify'])
        except TypeError:
            pass
    return tracks
  

In [6]:
tracks = extract_songs('https://open.spotify.com/playlist/37i9dQZEVXbqgPCgwknZJW?si=ae7b2ccfd8d64357')

In [7]:
tracks

['https://open.spotify.com/track/30yNdeqvvrMEbHjH4yA2Xk',
 'https://open.spotify.com/track/6y5UMIVVJDsULZwfZ0bNRX',
 'https://open.spotify.com/track/34YfyXjr42nealgrGr7Zll',
 'https://open.spotify.com/track/3H25rKGfEcqxqa5wFEexBS',
 'https://open.spotify.com/track/4GdvTdL7T1w1RoQlMBmC8k',
 'https://open.spotify.com/track/3o9bSWAYQv4RQ5umTeengp',
 'https://open.spotify.com/track/1Bqmsx21uoPYSUqU3n2k1F',
 'https://open.spotify.com/track/5SziiUUBWTSBpHUwtMpIp4',
 'https://open.spotify.com/track/02nhDSWvcXYALyVkth2oXd',
 'https://open.spotify.com/track/6qqq5zPYV7U4UsKnGC82tw',
 'https://open.spotify.com/track/0gWrMbx6pbdH3n3nsLjE55',
 'https://open.spotify.com/track/3qMimm5bezPaKIISP9f7mu',
 'https://open.spotify.com/track/055zEZwXYe70Z9ulw5JaSM',
 'https://open.spotify.com/track/0ETzlWlMrNumYZQDGzru0q',
 'https://open.spotify.com/track/1f1cI1rxKrMCLr5pD8rjX3',
 'https://open.spotify.com/track/1vuaMYsxmEARcAcnKEDoYT',
 'https://open.spotify.com/track/20blT6R5DJhfrk9c16lQ4E',
 'https://open

Then I need a function to get songs straight from a playlist, convert to feature info and put into a db

Let's have a function that will show stats for the playlist

Have a Map to Show Where 